In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install flask
!pip install pyngrok
!pip install boto3

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9m6muafs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-9m6muafs
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-m

In [ ]:
!pip install openai

In [ ]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
from torch import tensor,argmax
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
import boto3
from boto3.session import Session
import getpass
import os
import threading

from flask import Flask
from pyngrok import ngrok, conf


In [ ]:
#For generation of questions
tokenizer_q = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_q = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

#For generation of answers
model_a = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer_a = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model 

In [ ]:
!pip install openai-whisper

In [ ]:
import whisper

In [ ]:
# class temp:
#     def __init__(self, question, options):
#         self.question = question
#         self.options = options

In [ ]:
import re

def clean_transcript(transcript):
  cleaned = re.sub('[^A-Za-z0-9.]+', ' ',transcript)
  return cleaned

In [ ]:
def quiz_generator(transcript):
  #For questions
  def get_questions(context, max_length=64):
    qns=[]
    sentences=context.split('.')
    for sentence in sentences[:-1]:
        input_text = "answer: %s  context: %s </s>" % ('', sentence)
        features = tokenizer_q([input_text], return_tensors='pt')

        output = model_q.generate(input_ids=features['input_ids'],
                   attention_mask=features['attention_mask'],
                   max_length=max_length)
        qns.append(tokenizer_q.decode(output[0]).replace('<pad> question: ','').replace('</s>',''))
    return qns


    #For correct answer and 3 other options

  quiz_data = {}

  context_q_a = transcript
  # print("\nGenerated Question and Answers",end='\n\n')
  question_q_a=list(set(get_questions(context_q_a)))
  answer_q_a=[]
  for qn in question_q_a:
      answer_q_a.append(answer_question_with_options(qn,context_q_a))
  for i in range(len(question_q_a)):
      quiz_data[question_q_a[i]] = answer_q_a[i]
      # print('Qn.'+str(i+1)+'  '+question_q_a[i],sep='\n')
      # print(answer_q_a[i],sep='\n',end='\n\n')

  return quiz_data

In [ ]:
 def answer_question_with_options(question, context, num_options=3):
      input_ids = tokenizer_a.encode(question, context)
      sep_index = input_ids.index(tokenizer_a.sep_token_id)
      num_seg_a = sep_index + 1
      num_seg_b = len(input_ids) - num_seg_a
      segment_ids = [0]*num_seg_a + [1]*num_seg_b
      assert len(segment_ids) == len(input_ids)
      outputs = model_a(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

      start_scores = outputs.start_logits
      end_scores = outputs.end_logits

      # Find top start and end positions
      answer_start = torch.argmax(start_scores)
      answer_end = torch.argmax(end_scores)


      tokens = tokenizer_a.convert_ids_to_tokens(input_ids)

      #finding correct answer
      correct_answer = tokens[answer_start]
      for i in range(answer_start + 1, answer_end + 1):
          if tokens[i][0:2] == '##':
              correct_answer += tokens[i][2:]
          else:
              correct_answer += ' ' + tokens[i]


      # Get the other 3 options
      start_scores_sorted, start_indices_sorted = torch.topk(start_scores, num_options)
      end_scores_sorted, end_indices_sorted = torch.topk(end_scores, num_options)

      options = []

      options.append(correct_answer)

      # Generate options
      for i in range(num_options):
          start_index = start_indices_sorted[0][i].item()
          end_index = end_indices_sorted[0][i].item()
          tokens = tokenizer_a.convert_ids_to_tokens(input_ids[start_index:end_index + 1])
          option = ' '.join(tokens)
          options.append(option)

      return options

In [ ]:
def getmusic(key_id):
  your_access_key_id = ""     #Enter your aws access key id
  your_aws_access_key = ""    #Enter your aws secret access key
  session=Session(aws_access_key_id=your_aws_access_key_id, aws_secret_access_key=your_aws_secret_access_key)
  s3=session.resource('s3')
  mybucket=s3.Bucket("edtech-processed")
  mybucket.download_file("{}/output_audio.mp3".format(key_id),"/content/sample_data/{}.mp3".format(key_id))


In [ ]:
model = whisper.load_model("base")

In [ ]:
def transcribeAudio(key_id):
  result = model.transcribe("/content/sample_data/{}.mp3".format(key_id))
  with open("/content/{}.txt".format(key_id), "w") as f:
    f.write(result["text"])

In [ ]:

def func(key_id):
    path = "/content/{}.txt".format(key_id)
    tp = ""
    with open(path, 'r') as file:
      file_contents = file.read()
      tp+=file_contents

    if(len(tp)>400):
      tp = tp[:len(tp)-400]

    tp = clean_transcript(tp)
    quiz = quiz_generator(tp)

    return quiz

SERVER

In [ ]:
!pip install pyngrok

In [ ]:
pip install Flask-CORS


In [ ]:
from flask import Flask
from flask import Flask, request
from pyngrok import ngrok
from flask_cors import CORS

In [ ]:
port_no = 5000

In [ ]:
app = Flask(__name__)
CORS(app)

#Here you need to store your token from ngrok in the variable auth_token
auth token = ""
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(port_no).public_url

@app.route("/")
def home():
  return "hello"

@app.route("/process",methods=['POST'])
def index():
    data = request.get_json()
    print(data)

    # Check if 'keyid' and 'access_key' are in the request
    if 'name' in data:
      print(data['name']) #for confirming request is received or not

    if 'keyid' in data:
        keyid = data['keyid']
        getmusic(keyid)
        transcribeAudio(keyid)
        quiz = func(keyid)

        # Return the response as JSON
        return (quiz), 200
    else:
        # If 'keyid' or 'access_key' is missing, return an error
        error_message = {'error': 'Missing keyid or access_key in request'}
        return (error_message), 400

print(public_url)

app.run(port = port_no)